In [1]:
# loading dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = 'qanastek/51-languages-classifier'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

dataset = load_dataset("cakiki/rosetta-code",split='train')

In [2]:
# Extract the 'code' and 'language_name' columns
filtered_dataset = dataset.select_columns(['code','language_name'])

In [3]:
# filtered_dataset = filtered_dataset.select([0])

In [4]:
filtered_dataset

Dataset({
    features: ['code', 'language_name'],
    num_rows: 79013
})

In [5]:
from datasets import load_dataset
import datasets

# Load the cakiki/rosetta-code dataset
# dataset = load_dataset("cakiki/rosetta-code", "python")

# Function to split code snippets into chunks and associate them with the language name
def split_and_associate_language(example):
    max_token_limit = 510
    code = example["code"]
    language_name = example["language_name"]
    
    # Tokenize the code
    tokens = tokenizer.encode(code, add_special_tokens=True)
    # print(len(tokens))

    # Split into chunks
    chunks = [tokens[i:i + max_token_limit] for i in range(0, len(tokens), max_token_limit)]
    # print(len(chunks))

    # Create a list of dictionaries, each representing a chunk
    # chunked_examples = [{"code": tokenizer.decode(chunk, skip_special_tokens=True), "language_name": language_name} for chunk in chunks]
    chunked_examples = {'code':[],'language_name':[]}
    # code = []
    # language = []
    for chunk in chunks:
        chunked_examples['code'].append(tokenizer.decode(chunk, skip_special_tokens=True))
        chunked_examples['language_name'].append(language_name)
        # code += [tokenizer.decode(chunk, skip_special_tokens=True)]
        # language += [language_name]
    # print(chunked_examples)
    # print(language)
    

    return chunked_examples#{'code':code,'language_name':language}#chunked_examples


filtered_dataset = filtered_dataset.map(split_and_associate_language)

flattened_dataset = {
    'code': [item for sublist in filtered_dataset['code'] for item in sublist],
    'language_name': [item for sublist in filtered_dataset['language_name'] for item in sublist],
}
new_dataset = datasets.Dataset.from_dict({'code': flattened_dataset['code'], 'language_name': flattened_dataset['language_name']})
new_dataset

Map:   0%|          | 0/79013 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['code', 'language_name'],
    num_rows: 100816
})

In [20]:
new_dataset.save_to_disk('preprocessed_programming_language_data')

Saving the dataset (0/1 shards):   0%|          | 0/100816 [00:00<?, ? examples/s]

In [19]:
new_dataset.to_csv('preprocessed_programming_language_data.csv')

Creating CSV from Arrow format:   0%|          | 0/101 [00:00<?, ?ba/s]

61112666

In [15]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.Series(new_dataset['language_name']).value_counts()

#### Data Observation

In [8]:
data = pd.DataFrame(list(zip(dataset['code'],dataset['language_name'])),columns = ['code','language_name'],index = None)
# pd.DataFrame(list(zip(lst, lst2)),
#                columns =['Name', 'val'])
data['code'] = data['code'].apply(lambda x: x.replace('\n',' '))
data

,code,language_name
0,BEGIN # find all primes with strictly increasi...,ALGOL 68
1,ascending?: function [x][ initial: digits ...,Arturo
2,# syntax: GAWK -f ASCENDING_PRIMES.AWK BEGIN...,AWK
3,// Ascending primes. Nigel Galloway: April 1...,F#
4,USING: grouping math math.combinatorics math.f...,Factor
...,...,...
79008,"do(2){ask(""A B: "").split("" "").filter().sum().p...",zkl
79009,program: a_plus_b input: '7 11' output: 1...,Zoea
79010,"module ABProblem; var \ta,b: integer; begin ...",Zoea Visual
79011,"module ABProblem; var \ta,b: integer; begin ...",zonnon


In [9]:
data = pd.DataFrame(list(zip(new_dataset['code'],new_dataset['language_name'])),columns = ['code','language_name'],index = None)
# pd.DataFrame(list(zip(lst, lst2)),
#                columns =['Name', 'val'])
data['code'] = data['code'].apply(lambda x: x.replace('\n',' '))
data

,code,language_name
0,BEGIN # find all primes with strictly increasi...,ALGOL 68
1,newline ) ) FI OD END,ALGOL 68
2,ascending?: function [x][ initial: digits x an...,Arturo
3,# syntax: GAWK -f ASCENDING_PRIMES.AWK BEGIN {...,AWK
4,// Ascending primes. Nigel Galloway: April 19t...,F#
...,...,...
100811,"do(2){ask(""A B: "").split("" "").filter().sum().p...",zkl
100812,program: a_plus_b input: '7 11' output: 18,Zoea
100813,"module ABProblem; var a,b: integer; begin read...",Zoea Visual
100814,"module ABProblem; var a,b: integer; begin read...",zonnon
